In [6]:
import numpy as np
import pandas as pd

# load in data
data = pd.read_csv("card_transdata.csv")

#data.head()
df = pd.DataFrame(data)

In [ ]:
# clean up df

# remove rows where there is a missing value
df = df.dropna(axis=0)

